In [ ]:
%matplotlib notebook
import matplotlib
import matplotlib.pylab as plt
import scipy.ndimage.filters
import imagex
import ipywidgets
import IPython.display


In [ ]:
# load image
im = imagex.ImageData()
im.load_image("Image_445.sxm")

# display available channels
im.channel_names

In [ ]:
#plot frequency shift image
im.plot_image("Frequency_Shift");


In [ ]:
# get 2D image data
data_df = im.get_channel("Frequency_Shift")['data']
unit_df = im.get_channel("Frequency_Shift_fwd")['data_header']['unit']
data_current = im.get_data("Current")  # just an alternative way
data_current *= 1e12  # use pA

#plot raw data
fig, axs = plt.subplots(1,2)
im.plot_data(data_df, name="df", unit="Hz", axes=axs[0], no_labels=True);
im.plot_data(data_current, name="I", unit="pA", axes=axs[1], no_labels=True);
fig.tight_layout()

In [ ]:
# adjust color scale of the images interactively
imagex.images_colorscale_sliders(axs)


In [ ]:
# interactive gaussian laplace filter
fig2, ax2 = plt.subplots(1,1, figsize=(3,4.5))

@ipywidgets.interact(laplace=True, sigma=(0.0,0.1,0.001))
def plot_images(laplace=True, sigma=0.03):
    data =  data_df[:-30,:]
    if laplace:
        if sigma>0:
            gauss_sigma_px = im.nm_to_pixels(sigma)  # gaussian sigma is provided in rel units (nm), then converted into pixel-units
            data = -scipy.ndimage.filters.gaussian_laplace(data, sigma=gauss_sigma_px, mode='nearest')
    im.plot_data(data, axes=ax2, extra_output=False, no_labels=True, pixel_units=True);  # as we cut some of the data, we need to use pixel units now

fig2.tight_layout()

In [ ]:
imagex.save_figure(fig, "example_figure.png", dpi=100)  # save whole figure
imagex.save_axes(ax2, "example_image.png", dpi=100)  # save only image on the axes
